In [ ]:
print("Hello world")

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.9.1 64-bit.

In [1]:
# baseado em - https://www.kaggle.com/leandrodoze/sentiment-analysis-in-portuguese
#imports
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import numpy as np


Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.9.1 64-bit.

In [2]:
# https://www.kaggle.com/luisfredgs/imdb-ptbr
dataset = pd.read_csv('/home/renato/Documentos/imdb-reviews-pt-br.csv')
dataset.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [3]:
df2 = pd.read_csv('./files test/Tweets_Mg.csv')
df2['Classificacao'].value_counts()

Positivo    3300
Neutro      2453
Negativo    2446
Name: Classificacao, dtype: int64

## Unir os dois dataframe para teste

In [4]:
rm_df2 = [x for x in list(df2.columns) if x not in ['Text','Classificacao']]

In [5]:
#deixa os dois df com o mesmo numero de colunas
df = dataset.drop(columns=['text_en','id'])
df2 = df2.drop(columns=rm_df2)

In [6]:
#deixa as colunas de sentimentos igual a do outro df
df['sentiment'] = df['sentiment'].str.replace('neg', 'Negativo')
df['sentiment'] = df['sentiment'].str.replace('pos', 'Positivo')
df

,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",Negativo
1,Este é um exemplo do motivo pelo qual a maiori...,Negativo
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",Negativo
3,Nem mesmo os Beatles puderam escrever músicas ...,Negativo
4,Filmes de fotos de latão não é uma palavra apr...,Negativo
...,...,...
49454,"Como a média de votos era muito baixa, e o fat...",Positivo
49455,O enredo teve algumas reviravoltas infelizes e...,Positivo
49456,Estou espantado com a forma como este filme e ...,Positivo
49457,A Christmas Together realmente veio antes do m...,Positivo


In [7]:
# concatena os dois df
df.columns = ['Text', 'Polaridade']
df2.columns = ['Text', 'Polaridade']
df_concat = pd.concat([df,df2])
df_concat

,Text,Polaridade
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",Negativo
1,Este é um exemplo do motivo pelo qual a maiori...,Negativo
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",Negativo
3,Nem mesmo os Beatles puderam escrever músicas ...,Negativo
4,Filmes de fotos de latão não é uma palavra apr...,Negativo
...,...,...
8194,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo
8195,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo
8196,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo
8197,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo


In [8]:
df_concat['Polaridade'].value_counts()

Positivo    27994
Negativo    27211
Neutro       2453
Name: Polaridade, dtype: int64

## Construindo o modelo

In [9]:
tweets = df_concat["Text"].values
#tweets

In [10]:
classes = df_concat["Polaridade"].values
classes

array(['Negativo', 'Negativo', 'Negativo', ..., 'Positivo', 'Positivo',
       'Positivo'], dtype=object)

In [11]:
# Agora, vamos treinar o modelo usando a abordagem Bag of Words e o algoritmo Naive Bayes Multinomial
#    - Bag of Words, na prática, cria um vetor com cada uma das palavras do texto completo da base,
#      depois, calcula a frequência em que essas palavras ocorrem em uma data sentença, para então
#      classificar/treinar o modelo
#    - Exemplo HIPOTÉTICO de três sentenças vetorizadas "por palavra" e classificadas baseada na
#      frequência de suas palavras:
#         {0,3,2,0,0,1,0,0,0,1, Positivo}
#         {0,0,1,0,0,1,0,1,0,0, Negativo}
#         {0,1,1,0,0,1,0,0,0,0, Neutro}
#    - Olhando para esses vetores, meu palpite é que as palavras nas posições 2 e 3 são as com maior
#      peso na determinação de a que classe pertence cada uma das três sentenças avaliadas
#    - A função fit_transform faz exatamente esse processo: ajusta o modelo, aprende o vocabulário,
#      e transforma os dados de treinamento em feature vectors, a.k.a. vetor com frequêcia das palavras

vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(tweets)

modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB()

## Testando o Modelo

In [8]:


# Vamos usar algumas frases de teste para fazer a classificação com o modelo treinado
testes = ["Bom dia, que Hoje  e sexta"]

freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['pos'], dtype='<U3')

In [12]:
# Validação cruzada do modelo. Neste caso, o modelo é dividido em 10 partes, treinado em 9 e testado em 1
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)
# Quão acurada é a média do modelo?
metrics.accuracy_score(classes, resultados)


0.7260050643449305

## Testando com outra base de dados

In [13]:
df = pd.read_csv('../Dados coletados/Twitter_streaming.csv', low_memory=False)
df.head()

,ID,USUARIO,TWEET,DATA_TWEET
0,1,barbarabenitesv,"@giucostaa_ mas isso é muito tu né, não é só n...",2020-10-22 11:29:23
1,2,GORLSZ,@Alule1810 @Data24comar @cadenainforma Está bi...,2020-10-22 11:29:23
2,3,Carito_Cabrera7,@romanfolli Idem tesorero??,2020-10-22 11:29:24
3,4,driego22,"também, tudo tem que ficar escrevendo agora",2020-10-22 11:29:25
4,5,mgeissler,♥️,2020-10-22 11:29:25


In [14]:
def classifica_tweets(tweet):
    tweet = [tweet]
    freq_testes = vectorizer.transform(tweet)
    return modelo.predict(freq_testes)

In [38]:
df_tweets = list(df['TWEET'])
for i in range(1000,1150):
    predict = classifica_tweets(df_tweets[i])
    print(f"{predict} - {df_tweets[i]}\n")

['Negativo'] - @NerdBoomer A única luta bem desenvolvida em meninas superpoderosas foi no filme, de resto eram as mesmas sequências de golpes por episódio.

['Negativo'] - @gustavogomez462 @Palmeiras Parabéns monstro continue assim com essa categoria e raça....

['Positivo'] - Voltar a correr é sempre muito bom!!! Para o corpo e para a alma... em Lagoa da Pampulha https://t.co/ENGqqNNxaL

['Positivo'] - suas escolhas, suas consequências

['Negativo'] - @CacicAjuricaba Nós sempre temos razão. O tempo está mostrando.

['Negativo'] - Couve flor pequena 5,50 quase tive um troço

['Positivo'] - Bom dia pessoal!

['Positivo'] - viu esse tweet poste uma mídia aleatória sua https://t.co/59XrixtjwU

['Negativo'] - todo dia eu falo que vou dormir cedo
todo dia eu falho nessa missão

['Neutro'] - E vamos de Samba!

Novembro é diversão......

Muita roda de samba!

#amosamba  #novembro  #familia  #amigos em Guar… https://t.co/p7rsNHXE5X

['Negativo'] - Ninguém

['Neutro'] - @Herezeq Jajajaja

['Pos

['Negativo'] - @Guilherme_kk Bizarro

['Negativo'] - @PerrazPfm @joaoamoedonovo “Ainnn, preciso reafirmar meu bolsonarismo”
“Já sei, vou criar um fake pra chamar de gad… https://t.co/jyLNIjuQcG

['Negativo'] - @bismillahrry ainda não :/

['Positivo'] - @felipecf_ Melhor forma sempre

['Positivo'] - Bom dia amigos #FicaMateus #aFazenda12 https://t.co/2aG6bryl1a

['Positivo'] - Dica do dia: Faça o melhor que puder. Seja o melhor que puder. O resultado virá na mesma proporção de seu esforço.

['Negativo'] - Mano eu to indo pra escola e o herik do meu lado falando pra crlh eu n gosto de conversa de manha n ele parece q n entende

['Positivo'] - Bom dia ♥️

['Positivo'] - O Brasil tem um presidente da república isolado que tem como único objetivo rivalizar partidos de esquerda e emisso… https://t.co/wl70RgQ7ci

['Positivo'] - Tentar assistir #PrettyLittleLiars  também pois irá sair do catálogo

['Positivo'] - Bom dia cambada

['Positivo'] - Nesse mundo que vivemos, infelizmente é isso mesmo

In [43]:
classifica_tweets('e que passou do tempo')[0]

'Negativo'

In [ ]:
df_tweets['polaridade'] = np.array([classifica_tweets(tweet)[0] for tweet in df['TWEET'].values.astype('U')])

In [ ]:
df_tweets

## ingnore daqui pra baixo

In [ ]:
def clean()

In [63]:
#df = df.drop(columns = ['DATA_TWEET'])
df.to_csv('./files test/tweets_classificados.csv')
#df['Classificacao'] = '-'
